# CART

In [1]:
pred = function(suby) {
  # Takes a vector as input and performs majority vote 
    
  vote = rep(0,nlevels)
  for (i in 1:nlevels) { vote[i] = sum(suby==ylevels[i]) }
  return(ylevels[which.max(vote)])
}

In [2]:
gini = function(suby,summary = FALSE) {
  # Computes the Gini index of a given vector
    
  if (!summary) suby = as.vector(summary(suby))
  obs = sum(suby)
  return(   1-(drop(crossprod(suby)))/(obs*obs)  )
}

In [3]:
splitrule = function(subx,suby) { 
  # Takes a vector of feature and a vector of target as inputs, returns the best split rule of that feature 
    
  subylen = length(suby)
  xvalues  = sort(unique(subx))  
  if (length(xvalues)>1) { 
    cutpoint = ( xvalues[1:(length(xvalues)-1)] + xvalues[2:length(xvalues)] )/2
    minimpurity = 1  # 
    for ( i in 1:length(cutpoint) ) {
      lefty = suby[subx>=cutpoint[i]]
      righty = suby[subx<cutpoint[i]]
      impurity = ( gini(lefty)*length(lefty) + gini(righty)*length(righty) )/subylen
      if (impurity<minimpurity) {
        minimpurity = impurity
        splitpoint = cutpoint[i]
      }
    }
  }else {
    splitpoint = xvalues
    minimpurity = gini(suby)
  }
  return(c(splitpoint,minimpurity))
}

In [4]:
splitrule_best = function(subx,suby) {
  # A revised version of the above function 'splitrule', saves space and time

  suby_length = length(suby)
  xvalues = sort(unique(subx))
  if (length(xvalues)>1) {
    intervals = cut(subx,xvalues,right = FALSE) 
    suby_splited = matrix(unlist(by(suby,intervals,summary)),ncol = nlevels, byrow = TRUE)
    suby_splited_left = matrix(apply(suby_splited,2,cumsum),ncol = nlevels)
    suby_splited_right = sweep(-suby_splited_left,2,as.vector(summary(suby)),FUN = "+")
    suby_splited_left_obs = apply(suby_splited_left,1,sum)
    suby_splited_right_obs = suby_length - suby_splited_left_obs
    impurity = NULL
    for (i in 1:(length(xvalues)-1) ) {
      impurity = c(impurity,(gini(suby_splited_left[i,],summary = TRUE)*suby_splited_left_obs[i]
                             + gini(suby_splited_right[i,],summary = TRUE)*suby_splited_right_obs[i])/suby_length)
    }
    minimpurity = min(impurity)
    splitpoint = xvalues[which.min(impurity)]
  } else {
    splitpoint = xvalues
    minimpurity = gini(suby)
  }
  return(c(splitpoint,minimpurity))
}

In [5]:
splitrule_random = function(subx,suby) {
  # Another revised version of the above function 'splitrule'. It's a bit extreme since it doesn't sort the input feature
  # and simply selects one value from that feature randomly as split point.
    
  suby_length = length(suby)
  subx_withoutmax = subx[subx!=max(subx)]
  if (length(subx_withoutmax)>0) {
    #splitpoint = sample(subx[subx!=max(subx)],1)
    splitpoint = subx_withoutmax[sample(length(subx_withoutmax),1)]
    suby_splited_left = suby[subx<=splitpoint]
    suby_splited_right = suby[subx>splitpoint]
    impurity = (gini(suby_splited_left)*length(suby_splited_left) + gini(suby_splited_right)*length(suby_splited_right))/suby_length
  } else{
    splitpoint = 0
    impurity = 666
  }
  return(c(splitpoint,impurity))
}

In [6]:
splitting = function(subx,suby,split,rf) { 
  # Given a matrix of features and a vector of target, returns the best split feature and corresponding split rule

  if (!rf) chosen_variable = 1:k
  if (rf == TRUE) chosen_variable = sample(1:k,round(sqrt(k)))
  if (split == "best")  temp = apply(subx[,chosen_variable],2,splitrule_best,suby=suby)
  if (split == "random") temp = apply(subx[,chosen_variable],2,splitrule_random,suby=suby)
  splitpoint = temp[1,]
  minimpurity = temp[2,]
  splitvariable = chosen_variable[which.min(minimpurity)] 
  splitpoint = splitpoint[which.min(minimpurity)]
  minimpurity = min(minimpurity)
  return(c(splitvariable,splitpoint,minimpurity))
}

In [7]:
buildTREE = function(x,y,split = "best",rf = FALSE ) {
  # Build a decision tree based on the inputs
   
  TREE = NULL
  index = 1:n
  tree = as.data.frame(list(leftnode=0,rightnode=0,splitvariable=0,splitpoint=0,obs=n,
                            pred=levels(y)[1],leaf=TRUE,begin=1,end=n))  

  cur = 1 # current node being analysed 
  nodes = 1 # number of total nodes of this tree
  while ((cur<=nodes) ) {
    beginC = tree$begin[cur]; endC = tree$end[cur]
    indexCurrent = index[beginC:endC]
    subx = x[indexCurrent,]
    suby = y[indexCurrent]
    impurityCurrent = gini(suby)

    if (impurityCurrent>0.1) {
      temp <- splitting(subx,suby,split,rf)
      if (temp[3]<impurityCurrent) {
        # If the current node can be splitted into two sub nodes, nodes adds 2, otherwise nodes stay unchanged
        # Sooner or later the value of [cur] will exceed the value of [nodes] and thus break the loop
        tree$splitvariable[cur] = temp[1]
        tree$splitpoint[cur] = temp[2]
        tree$leftnode[cur] = nodes+1
        tree$rightnode[cur] = nodes+2
        nodes = nodes +2
        tree$leaf[cur] = FALSE # 
        indexL = indexCurrent[subx[,tree$splitvariable[cur]] <= tree$splitpoint[cur]]
        indexR = indexCurrent[subx[,tree$splitvariable[cur]] >  tree$splitpoint[cur]]
        index[beginC:endC] <- c(indexL,indexR)
          
        predL = pred(y[indexL])
        predR = pred(y[indexR])
        nodeL = as.data.frame(list(leftnode=0,rightnode=0,splitvariable=0,splitpoint=0,obs=length(indexL),
                                   pred=predL,leaf=TRUE,begin = beginC, end = beginC+length(indexL)-1) )
        nodeR = as.data.frame(list(leftnode=0,rightnode=0,splitvariable=0,splitpoint=0,obs=length(indexR),
                                   pred=predR,leaf=TRUE,begin = beginC+length(indexL),end = beginC+length(indexCurrent)-1) )
        tree = as.data.frame(rbind(tree,nodeL,nodeR))
      }
    }
    cur = cur + 1 # cur is the iterator of the loop

  }

  TREE <- tree
  return(TREE)
}

# random forest

In [8]:
predict = function(TREE,newx) { 
  # Given a matrix of features, returns a vector of predicted targets
    
  subpredict = function(subx) { #给定一行观测的自变量，我预测它的因变量
    # Given an observation, returns a scalar of its predicted target
      
    row = 1 
    while(TRUE) {
      if (TREE$leaf[row]==TRUE) {
        predicted = TREE$pred[row]
        break
      }
      if (subx[TREE$splitvariable[row]] <= TREE$splitpoint[row]) {row = TREE$leftnode[row]}
      else {row = TREE$rightnode[row]}
    }
    return(predicted)
  }
  return(apply(newx,1,subpredict))
}

In [9]:
bagging = function(x,y,trees = 150,split = "best",rf = FALSE) {
  # Ensemble method. Build a forest of many trees based on the input x and y using bootstrap aggregation
    
  TREES = list()
  for (i in 1:trees) {
    bootstrap_index = sample(1:n,n,replace = TRUE)
    x_bagging = x[bootstrap_index,]
    y_bagging = y[bootstrap_index]
    tree = buildTREE(x_bagging,y_bagging,split,rf)
    TREES = c(TREES,list(tree)) 
  }
  return(TREES)
}

In [10]:
predict_ensemble = function(newx,TREES) { 
  # Given a matrix of input features and a forest, returns a vector of predicted targets
    
  subpredict_ensemble = function(subx,TREES){ 
    # Given an observation and a forest, returns a scalar of itspredicted target
      
    subpredict = function(TREE,subx) { 
      # Given an observation and one single tree, returns a scalar of its predicted target
      row = 1 
      while(TRUE) {
        if (TREE$leaf[row]==TRUE) {
          predicted = TREE$pred[row]
          break
        }
        if (subx[TREE$splitvariable[row]] <= TREE$splitpoint[row]) {row = TREE$leftnode[row]}
        else {row = TREE$rightnode[row]}
      }
      return(predicted)
    }
    sub_predicted = unlist(lapply(TREES,subpredict,subx))  # each observation gets many predicions
    return( names(summary(sub_predicted))[which.max(summary(sub_predicted))] ) # every tree votes for final prediction
  }
  apply(newx,1,subpredict_ensemble,TREES)
}

# testing

In [13]:
# use data to show how to build one single decision tree
library(rpart)
data(kyphosis)
x = kyphosis[,2:4]
y = kyphosis[,1]

# some parameters
ylevels = levels(y)
nlevels = length(ylevels)
n = length(y)
k = dim(x)[2]

In [14]:
buildTREE(x,y)
# returns a tree
# each row of this matrix stands for one node of the tree

leftnode,rightnode,splitvariable,splitpoint,obs,pred,leaf,begin,end
2,3,3,8,81,absent,FALSE,1,81
4,5,1,8,19,present,FALSE,1,19
6,7,3,14,62,absent,FALSE,20,81
0,0,0,0,2,absent,TRUE,1,2
8,9,3,5,17,present,FALSE,3,19
10,11,1,51,33,absent,FALSE,20,52
0,0,0,0,29,absent,TRUE,53,81
12,13,1,130,12,absent,FALSE,3,14
0,0,0,0,5,present,TRUE,15,19
0,0,0,0,12,absent,TRUE,20,31


In [ ]:
# use data LetterRecognition for testing 
# below is the data preprocessiong process
library(mlbench)
data(LetterRecognition)
LEN = rep(0,26)
for(i in 1:26) LEN[i] = sum(LetterRecognition[,1]==LETTERS[i])
sub = NULL
for(i in 1:26) sub = c(sub,sample(which(LetterRecognition[,1]==LETTERS[i]),LEN[i]/50))
N = length(sub)
N1 = sum(LEN)-N

DATATR = LetterRecognition[sub,]

DATA = LetterRecognition[-sub,]
LEN = rep(0,26)
for(i in 1:26) LEN[i] = sum(DATA[,1]==LETTERS[i])

sub = NULL
for(i in 1:26) sub = c(sub,sample(which(DATA[,1]==LETTERS[i]),LEN[i]/10))
DATATE = DATA[sub,]

x = DATATR[,-1]
y = DATATR[,1]
ylevels = levels(y)
nlevels = length(ylevels)
n = length(y)
k = dim(x)[2]
chosen_variable = 1:k

In [ ]:
tree = buildTREE(x,y)
system.time(baggingtrees = bagging(x,y))
system.time(baggingtrees2 = bagging(x,y,split = "random"))

system.time(predi2 = apply(DATATE[,-1],1,subpredict_ensemble,baggingtrees))
system.time(predi3 = apply(DATATE[,-1],1,subpredict_ensemble,baggingtrees2))
result = (mean(predi!=DATATE[,1]))
result2 = (mean(predi2!=DATATE[,1]))
result3 = (mean(predi3!=DATATE[,1]))
result
result2
result3